In [ ]:
#
# Script meant to test the ability to show brief "help" animations in the interactive graph panel
# ... animations still showing incorrect behavior
#
import polars as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

_base_network_ = '0'
_base_dir_     = '../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df        = pl.DataFrame(_lu_)
_relates_ = [('fm','to')]
g         = rt.createNetworkXGraph(df, _relates_)

# Provides colors based on community
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(g):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)

#_rtg_ = rt.interactiveGraphPanel(df, {'relationships':_relates_, 'node_color':_node_colors_}, w=800, h=500)
#_rtg_

In [ ]:
#_rtg_.setAnimation('<rect x="10" y="10" width="10" height="10" fill="blue"/>')

In [ ]:
#_rtg_.setAnimation('<text x="5" y="15" fill="black"> popStack </text>')

In [ ]:
from rtsvg.polars_force_directed_layout import PolarsForceDirectedLayout
_pfdl_ = PolarsForceDirectedLayout(g)

In [ ]:
# The first element is garbage ... so throw away a few of them...
k0, k1, k2 = _pfdl_.stressVector(k=0)[16:], _pfdl_.stressVector(k=1)[16:], _pfdl_.stressVector(k=2)[16:]
df0, df1, df2 = pl.DataFrame({'x':[i for i in range(len(k0))], 'y':k0}), pl.DataFrame({'x':[i for i in range(len(k1))], 'y':k1}), pl.DataFrame({'x':[i for i in range(len(k2))], 'y':k2})
_params_ = {'x_field':'x', 'y_field':'y'}
rt.tile([rt.xy(df0, **_params_), rt.xy(df1, **_params_), rt.xy(df2, **_params_)])

In [ ]:
#
# Copied from PolarsForceDirectedLayout() - modified (mostly naming) to match the formula within the following paper:
#
# Drawing Graphs to Convey Proximity: An Incremental Arrangement Method
# J.D. Cohen
# ACM Transactions on Computer-Human Interaction, Vol. 4, No. 3, September 1997, Pages 197–229.
#
self   = _pfdl_
S_i    = 0
g_s    = self.g_s[S_i]
df_pos = self.df_anim[S_i][0]
mu     = 1.0/(2.0*len(g_s.nodes()))
k      = 2.0
__dx__, __dy__ = (pl.col('x') - pl.col('x_right')), (pl.col('y') - pl.col('y_right'))
df_pos = df_pos.join(df_pos, how='cross') \
                .filter(pl.col('node') != pl.col('node_right')) \
                .with_columns((__dx__**2 + __dy__**2).sqrt().alias('d')) \
                .join(self.df_dist[S_i], left_on=['node', 'node_right'], right_on=['fm','to']) \
                .with_columns(pl.col('t').pow(k).alias('t_k')) \
                .with_columns(pl.when(pl.col('d') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('d')).alias('d'),
                              pl.when(pl.col('t') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('t')).alias('w')) \
                .with_columns(((2.0*__dx__*(1.0 - pl.col('t')/pl.col('d')))/pl.col('t_k')).alias('xadd'),
                              ((2.0*__dy__*(1.0 - pl.col('t')/pl.col('d')))/pl.col('t_k')).alias('yadd'),
                              (((pl.col('t') - pl.col('d'))**2)/pl.col('t_k')).alias('stress')) \
                .group_by(['node','x','y','s']).agg(pl.col('xadd').sum(), pl.col('yadd').sum(), pl.col('stress').sum()/len(g_s.nodes())) \
                .with_columns(pl.when(pl.col('s')).then(pl.col('x')).otherwise(pl.col('x') - mu * pl.col('xadd')).alias('x'),
                              pl.when(pl.col('s')).then(pl.col('y')).otherwise(pl.col('y') - mu * pl.col('yadd')).alias('y')) \
                .drop(['xadd','yadd'])
df_pos.filter(pl.col('node').is_in({'147','242','5'})).sort('node')